# Performance Analysis -  Numba
> Number of effective sequences implemented in Numba
- toc: true
- branch: master
- badges: true
- author: Donatas Repečka
- categories: [performance]

## Introduction

In [the previous post](https://donatasrep.github.io/donatas.repecka/performance/2021/04/27/Performance-comparison.html) I have compared various languages and libraries in terms of their speed. This notebook contains the code used in the comparison as well as some details about the choices made to improve the performance of numpy implementation.

## Setup

In [9]:
# !wget https://github.com/donatasrep/donatas.repecka/blob/master/data/picked_msa.fasta

In [2]:
# ! pip install numpy
# ! pip install pandas
# ! pip install numba

## Getting data

I will cheat here and use pandas to help me to read the file. 

In [15]:
import pandas as pd

In [18]:
def get_data(path):
    fasta_df = pd.read_csv(path, sep="\n", lineterminator=">", index_col=False, names=['id', 'seq'])
    return fasta_df.seq.to_numpy(dtype=str)

In [19]:
seqs = get_data('../data/picked_msa.fasta')

Just to remind the pseudo code looks like this:

```
for seq1 in seqs:
  for seq2 in seqs:
    if count_mathes(seq1, seq2) > threshold:
      weight +=1
  meff += 1/weight
 
meff = meff/(len(seq1)^0.5)
```

In [23]:
import numpy as np
from numba import jit, njit, prange

In [24]:
def get_nf_numba(seqs, threshold=0.8):
    seqs = seqs.view(np.uint32).reshape(seqs.shape[0], -1)
    n_seqs, seq_len = seqs.shape
    is_same_cluster = np.eye(n_seqs)
    for i in prange(n_seqs):
        c  = 0
        for j in prange(i+1, n_seqs):
            identity = np.equal(seqs[i], seqs[j]).mean()
            is_more = np.greater(identity, threshold)
            is_same_cluster[i,j] = is_more
            is_same_cluster[j,i] = is_more
    meff = 1.0/is_same_cluster.sum(1)
    return meff.sum()/(seq_len**0.5)

In [56]:
fn = jit(get_nf_numba, nopython=True,parallel=False)
fn(seqs[:10])

0.0765278731793381

In [57]:
%%timeit -n 3 -r 3
fn(seqs[:100])

11.1 ms ± 75.5 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [58]:
fn = jit(get_nf_numba, nopython=True,parallel=True)
fn(seqs[:10])

0.0765278731793381

In [59]:
%%timeit -n 3 -r 3
fn(seqs[:100])

4.9 ms ± 856 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [62]:
fn = jit(get_nf_numba, nopython=True,parallel=False, fastmath=True)


0.0765278731793381

In [63]:
%%timeit -n 3 -r 3
fn(seqs[:100])

10.6 ms ± 39 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [64]:
fn = jit(get_nf_numba, nopython=True,parallel=True, fastmath=True)
fn(seqs[:10])

0.0765278731793381

In [65]:
%%timeit -n 3 -r 3
fn(seqs[:100])

4.09 ms ± 2.14 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
